In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = 'v0'
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_02'
    dataset_name = 'RNA_DatasetBaseline'
    
    model_name = 'CustomTransformerV1'
    dim = 192
    depth = 12
    dim_head = 32
   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

In [5]:
df = pd.read_parquet(os.path.join(CFG.path,'train_data.parquet'))
ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='train',sn_train=False)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='train', mask_only=True,sn_train=False)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='eval')
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
gc.collect()

3463

In [6]:
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)().cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    CSVLogger(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


epoch,train_loss,valid_loss,mae,time
0,0.145914,0.180683,0.193767,13:24
1,0.136295,0.163772,0.176454,13:29
2,0.132718,0.156711,0.168576,13:29
3,0.128044,0.150866,0.162853,13:31
4,0.126802,0.146537,0.158348,13:31
5,0.125488,0.143752,0.155057,13:33
6,0.123523,0.141605,0.153119,13:30
7,0.121411,0.139843,0.151121,13:29
8,0.122817,0.137761,0.148783,13:33
9,0.121368,0.138468,0.149656,13:31


Better model found at epoch 0 with mae value: 0.19376743160266147.
Better model found at epoch 1 with mae value: 0.17645419353175262.
Better model found at epoch 2 with mae value: 0.1685760341574818.
Better model found at epoch 3 with mae value: 0.16285331401238926.
Better model found at epoch 4 with mae value: 0.15834841700146243.
Better model found at epoch 5 with mae value: 0.15505702124257947.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 6 with mae value: 0.15311917786169923.
Better model found at epoch 7 with mae value: 0.15112086180270262.
Better model found at epoch 8 with mae value: 0.148783308060138.
Better model found at epoch 11 with mae value: 0.14660300900543138.
Better model found at epoch 12 with mae value: 0.14585531225989107.
Better model found at epoch 13 with mae value: 0.1455968401533539.
Better model found at epoch 14 with mae value: 0.1439263909799168.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 21 with mae value: 0.14081402334800835.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 24 with mae value: 0.13973140651257954.


KeyboardInterrupt: 